In [5]:
import pandas as pd


df = pd.read_csv('cleaned_ssml_dataset_2024080616.csv')

df.head()

,text,ssml
0,Under the soft glow of the vintage chandeliers...,<speak> Under the soft glow of the vintage ...
1,Captain Zara adjusted the spaceship's controls...,<speak> Captain Zara adjusted the spaceship...
2,"Under the soft glow of the café lights, Emma's...",<speak> Under the soft glow of the café lig...
3,"Sarah's alarm buzzed at precisely 6:30 AM, pul...",<speak> Sarah's alarm buzzed at precisely <sa...
4,"At precisely 6:30 AM, Mia's alarm blared throu...","<speak> At precisely <say-as interpret-as=""..."


In [1]:
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import soundfile as sf
import torch

device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"
if torch.backends.mps.is_available():
    device = "mps"
if torch.xpu.is_available():
    device = "xpu"
torch_dtype = torch.float16 if device != "cpu" else torch.float32


/home/stanley/anaconda3/envs/audio_p11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Flash attention 2 is not installed


In [2]:
device

'cuda:0'

In [3]:

# model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler_tts_mini_v0.1", torch_dtype=torch_dtype).to(device)

# tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler_tts_mini_v0.1")


model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler-tts-mini-expresso").to(device)
tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler-tts-mini-expresso")


/home/stanley/anaconda3/envs/audio_p11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/stanley/anaconda3/envs/audio_p11/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [6]:
test_text = df['text'][0]
print(test_text)

Under the soft glow of the vintage chandeliers at Café Vienna, nestled on 54th Elm Street, Sophie felt a flutter of nerves as she stirred her cinnamon latte. The café, with its cushioned booths and old world charm, was her favorite spot in town. Every corner was adorned with bookshelves and potted plants, creating an intimate, inviting atmosphere. 

She glanced at the door just in time to see Daniel walk in. His tall frame was familiar, but there was an easy grace in his steps she hadn’t noticed before. His dark hair was slightly tousled by the autumn wind, and his blue eyes lit up as they met hers. The background hum of hushed conversations seemed to fade away.

"Hey, Sophie," he greeted, his voice warm and sincere. He slid into the seat across from her, setting down a single red rose—a gesture that made her heart skip a beat.

“Daniel, you didn’t have to,” she said, her cheeks flushing a delicate pink as she accepted the flower. 

“I wanted to,” he replied, his gaze unwavering. “It’s

In [41]:
test_text = """
Under the soft glow of the vintage chandeliers at Café Vienna, nestled on 54th Elm Street, Sophie felt a flutter of nerves as she stirred her cinnamon latte.

"""

test_text = """

he conversation flowed effortlessly as memories and laughter bridged the gap of years. They recalled old stories, their lives weaving a tapestry of shared moments. Sophie realized how much she had missed 


"""

In [42]:
word_count = len(test_text.split())
print(word_count)

32


In [43]:
prompt = test_text
#description = "Jerry speaks with a confused tone and at a moderate speed with high quality audio, speaker with a slightly low-pitched voice delivers her words quite expressively he speaks slowly"# "A female speaker with a slightly low-pitched voice delivers her words quite expressively, in a very confined sounding environment with clear audio quality. **high quality audio**"
# description = "A female speaker that takes the SSML tags inside the text for reference, in a very confined sounding environment with clear audio quality."
description = """ 
gender": "female",

          "age": 35,

          "accent": "Southern",

          "emotionalIntensity": "calm",

          "pitch": "180 Hz",

          "speakingRate":
          
          """


In [44]:
input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)


In [47]:

generation = model.generate(input_ids=input_ids, 
                            prompt_input_ids=prompt_input_ids,
                            max_new_tokens=3000
                            ).to(torch.float32)
audio_arr = generation.cpu().numpy().squeeze()

In [48]:
sf.write("parler_tts_out_ssml_new.mp3", audio_arr, model.config.sampling_rate)